In [1]:
import os
import re
import zipfile
import urllib, urllib2
import pandas as pd, numpy as np
from bs4 import BeautifulSoup
import glob
import time
from collections import OrderedDict

Download files for all states

In [2]:
def extract_zipped_files(sequence,archv):    
    filenames=[]
    new_sequence=[]
    for elem in sequence:
        new_sequence.append(elem+'2010'+'.sf1')
    print new_sequence
    with zipfile.ZipFile(archv, "r") as z:  
        # Look for the file we need
        for txt_file in z.namelist():
            if txt_file.endswith(tuple(new_sequence)) or ('geo' in txt_file):
                z.extract(txt_file,path='./states/')

In [3]:
sequence=['044']

In [17]:
states=['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
        'District_of_Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
        'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
        'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New_Hampshire', 'New_Jersey', 'New_Mexico', 'New_York',
        'North_Carolina', 'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto_Rico', 'Rhode_Island',
        'South_Carolina', 'South_Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
        'West_Virginia', 'Wisconsin', 'Wyoming']


tags=[]
states_short={}
for state in states:
#     print state
    url="https://www2.census.gov/census_2010/04-Summary_File_1/{}/".format(state)
    soup = BeautifulSoup(urllib2.urlopen(url))
    tag=soup.find_all(href=re.compile("\.zip"))[0]
    t_state=tag['href']
    state_file_dl = urllib.URLopener()
    state_file_dl.retrieve(url+t_state,os.path.join('states', t_state))
    extract_zipped_files(sequence,os.path.join('states', t_state))
    states_short[state]=t_state[0:2]
    time.sleep(20)

In [19]:
category_xref = OrderedDict([('own1','h0160003'),('own2','h0160004'),( 'own3','h0160005'),('own4','h0160006'),('own5','h0160007'),
             ('own6','h0160008'),('own7','h0160009'),('rent1','h0160011'),('rent2','h0160012'),('rent3','h0160013'),
             ('rent4','h0160014'),('rent5','h0160015'),('rent6','h0160016'),('rent7','h0160017'),
             ('residential_units','h0030001')])

In [23]:
def get_county_file(state_long,abb,seq,category_xref,path):
    print state_long
    col_names = ['fileid','stusab','sumlev','geocomp','chariter','cifsn','logrecno','region','division','state','county','countycc','countysc','cousub','cousubcc','cousubsc','place','placecc','placesc','tract','blkgrp','block','arealand','puma']

    colspecs =[(0,6),(6,8),(8,11),(11,13),(13,16),(16,18),(18,25),(25,26),(26,27),(27,29),(29,32),(32,34),(34,36),(36,41),(41,43),(43,45),(45,50),(50,52),(52,54),(54,60),(60,61),(61,66),(198,212),(477,482)]

    columns_to_drop = ['fileid','stusab','sumlev','geocomp','chariter','cifsn','region','division','countycc','countysc','cousub','cousubcc','cousubsc','place','placecc','placesc','puma']

    sf1_file = '{}geo2010.sf1'.format(abb) 
    state = sf1_file[:2]

    geo_df = pd.read_fwf(path+sf1_file, colspecs = colspecs,header = None,names = col_names)
    df_block = geo_df[geo_df.block > 0]
    df_block = df_block.drop(columns_to_drop,axis=1)

    state_file = "{}00{}2010.sf1".format(abb,seq) 
    df_seq = pd.read_csv(path+state_file ,header=None,prefix='X')
    df_seq = df_seq.drop(['X0','X1','X2','X3'],axis=1)
    df_merge = pd.merge(df_block,df_seq,left_on='logrecno',right_on='X4')
    df_merge = df_merge.drop(['X4'],axis=1)
    file_name = state_file.replace('.sf1', '.csv')
    for column in df_merge.columns:
        if df_merge['%s' % (column)].dtype == 'float64':
            df_merge['%s' % (column)] = df_merge['%s' % (column)].astype('int')
    if len(df_merge) > 0:
        df_merge.to_csv(path+file_name, index = False)

    f = "tx00{}_sf1.csv".format(seq)
    filenum = int(f[5:7])
    df = pd.read_csv(path+f, skiprows = 1)
    if df.columns[0] == 'SF1ST':
        df = pd.read_csv(path+f)
    cols = df.columns[6:]


    f = '{}00{}2010.csv'.format(abb,seq)
    filenum = int(f[5:7])
    df = pd.read_csv(path+f)
    new_columns = list(df.columns[:7].values) + list(cols.values)
    df.columns = new_columns
    df.to_csv(path+f, index = False)

    segment = pd.read_csv(path+f).set_index(['logrecno', 'state', 'county', 'tract', 'blkgrp', 'block', 'arealand'])

    df = segment.reset_index()
    df.state = df.state.astype('str').apply(lambda s: '{:>02}'.format(s))
    df.county = df.county.astype('str').apply(lambda s: '{:>03}'.format(s))
    df.tract = df.tract.astype('str').apply(lambda s: '{:>06}'.format(s))
    df.block = df.block.astype('str').apply(lambda s: '{:>04}'.format(s))
    df['blockid10'] = df.state + df.county + df.tract + df.block
    df.set_index('blockid10', inplace = True)

    cols = ["state", "county", "tract", 'blkgrp'] + category_xref.values()

    df.columns = [col.lower() for col in df.columns]

    df_subset = df[cols]



    category_xref2 = {y:x for x,y in category_xref.iteritems()}



    colnames = list(df_subset.columns[:4]) + [category_xref2[col] for col in df_subset.columns if 'h0' in col]

    df_subset.columns = colnames

    df_subset['state_county'] = df_subset.state + df_subset.county

    df_subset['total_own'] = df_subset.own1 + df_subset.own2 + df_subset.own3 + df_subset.own4 + df_subset.own5 + df_subset.own6 + df_subset.own7
    df_subset['total_rent'] = df_subset.rent1 + df_subset.rent2 + df_subset.rent3 + df_subset.rent4 + df_subset.rent5 + df_subset.rent6 + df_subset.rent7

    print(df_subset[['total_own', 'total_rent']].sum())

    df_subset.to_csv('./sf1_files/{}_sf1.csv'.format(state_long.lower())) 

In [24]:
for state,abb in states_short.items():
    for seq in sequence:
        print (state)
        path='./states/'
        get_county_file(state,abb,seq,category_xref,path)        

Mississippi
Mississippi


C:\Users\sol\Anaconda3\envs\metcouncil\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sol\Anaconda3\envs\metcouncil\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sol\Anaconda3\envs\metcouncil\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

total_own     777073
total_rent    338695
dtype: int64
Oklahoma
Oklahoma
total_own     981760
total_rent    478690
dtype: int64
Delaware
Delaware
total_own     246724
total_rent     95573
dtype: int64
Minnesota
Minnesota
total_own     1523859
total_rent     563368
dtype: int64
Illinois
Illinois
total_own     3263639
total_rent    1573333
dtype: int64
Arkansas
Arkansas
total_own     768156
total_rent    378928
dtype: int64
Indiana
Indiana
total_own     1747975
total_rent     754179
dtype: int64
Maryland
Maryland
total_own     1455775
total_rent     700636
dtype: int64
Louisiana
Louisiana
total_own     1162299
total_rent     566061
dtype: int64
District_of_Columbia
District_of_Columbia
total_own     112055
total_rent    154652
dtype: int64
Idaho
Idaho
total_own     404903
total_rent    174505
dtype: int64
New_York
New_York
total_own     3897837
total_rent    3419918
dtype: int64
West_Virginia
West_Virginia
total_own     561013
total_rent    202818
dtype: int64
Tennessee
Tennessee
total_o